In [1]:
from typing import Dict
import os
from glob import glob
from tqdm.auto import tqdm
import pandas as pd
import shutil

In [2]:
edge_reader_model = """let edges_reader = EdgeFileReader::new("{target_path}")?
        .set_rows_to_skip({rows_to_skip})
        .set_header({header})
        .set_separator({separator})?
        .set_verbose({verbose})
        .set_max_rows_number({max_rows_number})
        .set_sources_column({sources_column})?
        .set_sources_column_number({sources_column_number})?
        .set_destinations_column({destinations_column})?
        .set_destinations_column_number({destinations_column_number})?
        .set_weights_column({weights_column})?
        .set_weights_column_number({weights_column_number})?
        .set_default_weight({default_weight})
        .set_ignore_duplicates({ignore_duplicates})
        .set_skip_selfloops({skip_selfloops})
        .set_numeric_edge_type_ids({numeric_edge_type_ids})
        .set_numeric_node_ids({numeric_node_ids})
        .set_default_edge_type({default_edge_type}))
        .set_skip_weights_if_unavailable({skip_weights_if_unavailable})
        .set_skip_edge_types_if_unavailable({skip_edge_types_if_unavailable})
        .set_edge_types_column({edge_types_column})?
        .set_edge_types_column_number({edge_types_column_number})?"""

In [3]:
node_reader_model = """let nodes_reader = Some(NodeFileReader::new("{target_path}")?
        .set_rows_to_skip({rows_to_skip})
        .set_separator({separator})?
        .set_header({header})
        .set_verbose({verbose})
        .set_ignore_duplicates({ignore_duplicates})
        .set_default_node_type({default_node_type})
        .set_node_types_separator({node_types_separator})?
        .set_nodes_column({nodes_column})?
        .set_nodes_column_number({nodes_column_number})
        .set_node_types_column({node_types_column})?
        .set_node_types_column_number({node_types_column_number})"""

In [4]:
default_node_reader_model = """let nodes_reader = None;"""

In [5]:
regression_test_model = """extern crate graph;

use graph::{{{useful_imports}}};

#[test]
/// This is a regression test that has been automatically generated
/// by the fuzzer harness.
/// The test originally caused a panic in the file {filename},
/// specifically (at the time) line {line_number} and column {column_number}.{message}
///
fn test_regression_{current_test_id}() -> Result<(), String> {{
    {edges_reader}

    {nodes_reader}

    let mut graph = Graph::from_unsorted_csv(
        edges_reader,
        nodes_reader,
        {directed}, // Directed
        {directed_edge_list}, // Directed edge list
        "Fuzz Graph" // Name of the graph
    )?;
    let _ = graph::test_utilities::default_test_suite(&mut graph, Some(false));
    Ok(())
}}
"""

In [6]:
def load_metadata(path:str)->Dict:
    print(path)
    with open(path, "r") as f:
        return dict([
            (line.strip().split(",", 1)[0], "")
            if line.strip().split(",", 1)[1] == ""
            else line.strip().split(",", 1)
            for line in f.readlines()
            if "," in line
        ])

def format_constructor(model, metadata_path, current_test_id, list_type) -> str:
    if list_type not in ("edges", "nodes"):
        raise ValueError("Given list type is not supported!")
    
    # Move the new test edge list
    target_path = os.path.join(
        "tests/data/regression/"
        "{}.{}".format(current_test_id, list_type)
    )
    
    file_metadata = load_metadata(
        metadata_path
    )
    
    
    # Remove all None values
    simplified_model = "\n".join([
        line
        for line in model.split("\n")
        if not any(
            "set_{}(".format(param) in line
            for param, value in file_metadata.items()
            if value == "None"
        )
    ])
    
    if list_type == "nodes":
        simplified_model += ")"
    
    simplified_model += ";"
        
    return simplified_model.format(
        target_path=target_path,
        **file_metadata
    ), target_path, file_metadata

In [9]:
move = True


os.makedirs("../graph/tests/data/regression", exist_ok=True)
for test_directory in tqdm(
    glob("unit_tests/*"),
    desc="Building regression tests"
):
    edges_path = os.path.join(
        test_directory,
        "edges.edges"
    )
    
    # If the edge list is empty we skip forwards
    with open(edges_path, "r") as f:
        if len(f.readlines()) == 0:
            continue
    
    edges_metadata_path = os.path.join(
        test_directory,
        "edges_metadata.csv"
    )
    report_path = os.path.join(
        test_directory,
        "report.txt"
    )
    
    if os.path.exists(report_path):
        report = open(report_path, "r").read()
    else:
        report = "Report not provided"
    
    graph_metadata = load_metadata(os.path.join(
        test_directory,
        "graph_metadata.csv"
    ))
    panic_path = os.path.join(
        test_directory,
        "panic.csv"
    )
    if not os.path.exists(panic_path):
        continue
    
    panic_metadata = load_metadata(panic_path)
    
    usefull_imports = ["Graph", "EdgeFileReader"]
    
    has_node_file = any(
        "node" in file
        for file in os.listdir(test_directory)
    )
    
    if has_node_file:
        nodes_path = os.path.join(
            test_directory,
            "nodes.nodes"
        )
        with open(nodes_path, "r") as f:
            has_node_file = len(f.readlines()) > 0
    
    current_test_id = max([0, *[
        int(test_name.split(".")[0])
        for test_name in os.listdir("../graph/tests/data/regression")
    ]]) + 1
    
    if has_node_file:
        usefull_imports.append("NodeFileReader")
        nodes_path = os.path.join(
            test_directory,
            "nodes.nodes"
        )
        nodes_metadata_path = os.path.join(
            test_directory,
            "nodes_metadata.csv"
        )
        nodes_reader, target_node_path, _ = format_constructor(
            node_reader_model,
            nodes_metadata_path,
            current_test_id,
            "nodes"
        )
        if move:
            os.rename(
                nodes_path,
                os.path.join(
                    "../graph",
                    target_node_path
                )
            )
    else:
        nodes_reader = default_node_reader_model

    edges_reader, target_edge_path, metadata = format_constructor(
        edge_reader_model,
        edges_metadata_path,
        current_test_id,
        "edges"
    )

    if move:
        os.rename(
            edges_path,
            os.path.join(
                "../graph",
                target_edge_path
            )
        )
        
    regression_test = regression_test_model.format(
        edges_reader=edges_reader,
        nodes_reader=nodes_reader,
        current_test_id=current_test_id,
        filename=panic_metadata["file"].split(os.sep)[-1].strip('"'),
        line_number=panic_metadata["line"],
        column_number=panic_metadata["col"],
        message="" if "message" not in panic_metadata else "\n/// The provided message was: '{}'".format(panic_metadata["message"]),
        useful_imports=", ".join(usefull_imports),
        report=report,
        **graph_metadata
    )
    
    with open("../graph/tests/test_regression_{}.rs".format(current_test_id), "w") as f:
        f.write(regression_test)
    
    if move:
        shutil.rmtree(test_directory)

Building regression tests:   0%|          | 0/1340 [00:00<?, ?it/s]

unit_tests/7dwuVGqzIvZci9igdWZpIpRj54vXHj1aN847MtMiVkIWJjXDT83s61LWYe5hNVyU/graph_metadata.csv
unit_tests/7dwuVGqzIvZci9igdWZpIpRj54vXHj1aN847MtMiVkIWJjXDT83s61LWYe5hNVyU/panic.csv
unit_tests/7dwuVGqzIvZci9igdWZpIpRj54vXHj1aN847MtMiVkIWJjXDT83s61LWYe5hNVyU/edges_metadata.csv
unit_tests/imcU93dMEiIghWVZQQiQS5ffsfWildQdxqF269U9RdsvkSBSeVVKNU6kROlIfI9w/graph_metadata.csv
unit_tests/imcU93dMEiIghWVZQQiQS5ffsfWildQdxqF269U9RdsvkSBSeVVKNU6kROlIfI9w/panic.csv
unit_tests/imcU93dMEiIghWVZQQiQS5ffsfWildQdxqF269U9RdsvkSBSeVVKNU6kROlIfI9w/edges_metadata.csv
unit_tests/zb240zcITmOx9zaC5BKMCbuYkthIXedqCn5nJBr0i1NovLiM67ntTJ3i3WfYSrYV/graph_metadata.csv
unit_tests/zb240zcITmOx9zaC5BKMCbuYkthIXedqCn5nJBr0i1NovLiM67ntTJ3i3WfYSrYV/panic.csv
unit_tests/zb240zcITmOx9zaC5BKMCbuYkthIXedqCn5nJBr0i1NovLiM67ntTJ3i3WfYSrYV/edges_metadata.csv
unit_tests/pFEsMgD5UJ4TcfILB0AScmqTJ9jVyI6t5ZPvvp87WQuS1QmQ9T40uuL5sm2TDJUa/graph_metadata.csv
unit_tests/pFEsMgD5UJ4TcfILB0AScmqTJ9jVyI6t5ZPvvp87WQuS1QmQ9T40uuL5sm2TDJUa/p

unit_tests/K6Wu1AuoD6c32WgNqTBNRv3wa1ZrO533L7qLb9ZLhekc1vHJKLkmZuDCja0ISHp4/graph_metadata.csv
unit_tests/K6Wu1AuoD6c32WgNqTBNRv3wa1ZrO533L7qLb9ZLhekc1vHJKLkmZuDCja0ISHp4/panic.csv
unit_tests/K6Wu1AuoD6c32WgNqTBNRv3wa1ZrO533L7qLb9ZLhekc1vHJKLkmZuDCja0ISHp4/edges_metadata.csv
unit_tests/KbBQfJeCACm0kqW2GzKj11amAArVPsq3rgpyCXVDFYWIAVQ3LK2iebV9B1HTJVYH/graph_metadata.csv
unit_tests/KbBQfJeCACm0kqW2GzKj11amAArVPsq3rgpyCXVDFYWIAVQ3LK2iebV9B1HTJVYH/panic.csv
unit_tests/KbBQfJeCACm0kqW2GzKj11amAArVPsq3rgpyCXVDFYWIAVQ3LK2iebV9B1HTJVYH/edges_metadata.csv
unit_tests/2bjpxgIihfF82lWiEm0frrWYxirtYygcpRtxykZXrlUtRI3LvCUDWCREc07prjqc/graph_metadata.csv
unit_tests/2bjpxgIihfF82lWiEm0frrWYxirtYygcpRtxykZXrlUtRI3LvCUDWCREc07prjqc/panic.csv
unit_tests/2bjpxgIihfF82lWiEm0frrWYxirtYygcpRtxykZXrlUtRI3LvCUDWCREc07prjqc/edges_metadata.csv
unit_tests/psRiodIE7IcTjDFObLHxqtOJN9C34yaKlwO4SV9uLuKg9QgOigXAUJUpLn5wbuOj/graph_metadata.csv
unit_tests/psRiodIE7IcTjDFObLHxqtOJN9C34yaKlwO4SV9uLuKg9QgOigXAUJUpLn5wbuOj/p

unit_tests/gNLVsy9xiBFOFQ8LdssAkauat22vAJkrvlOBuhWRuWAjyXtk3eiTIJ173FUHlPGQ/edges_metadata.csv
unit_tests/CqGX2NWZEOd7suXShxJeIbl8HYlmPbHtoQdQobm8xxMgucWJQx4G3N9CO1P7PI8G/graph_metadata.csv
unit_tests/CqGX2NWZEOd7suXShxJeIbl8HYlmPbHtoQdQobm8xxMgucWJQx4G3N9CO1P7PI8G/panic.csv
unit_tests/CqGX2NWZEOd7suXShxJeIbl8HYlmPbHtoQdQobm8xxMgucWJQx4G3N9CO1P7PI8G/edges_metadata.csv
unit_tests/KJmiC3gmSL1g7BLsq4gA1cFz6FyhXNmkNNF6oguHGk2XuPkzmZI8xFuJSwOhyu7x/graph_metadata.csv
unit_tests/KJmiC3gmSL1g7BLsq4gA1cFz6FyhXNmkNNF6oguHGk2XuPkzmZI8xFuJSwOhyu7x/panic.csv
unit_tests/KJmiC3gmSL1g7BLsq4gA1cFz6FyhXNmkNNF6oguHGk2XuPkzmZI8xFuJSwOhyu7x/edges_metadata.csv
unit_tests/kzcSztdksu8JOpD20QWQtcOvclcjszSM3Ak9KbKGZOKcu8GpGUS0S48Eg8knX7hU/graph_metadata.csv
unit_tests/kzcSztdksu8JOpD20QWQtcOvclcjszSM3Ak9KbKGZOKcu8GpGUS0S48Eg8knX7hU/panic.csv
unit_tests/kzcSztdksu8JOpD20QWQtcOvclcjszSM3Ak9KbKGZOKcu8GpGUS0S48Eg8knX7hU/edges_metadata.csv
unit_tests/EOLWK5yiD7WQtqeAdQebfy81jwt5lq5RapmDbZVbEjyzm8HxDdBnhy9SkAXaE3XF/g

unit_tests/GFfQbPv6Lv3BvKTQsb6lvo1Yrs9rebvj3tVpWD8tJ0GtifAJLpAoX0l8x1CCJEly/graph_metadata.csv
unit_tests/GFfQbPv6Lv3BvKTQsb6lvo1Yrs9rebvj3tVpWD8tJ0GtifAJLpAoX0l8x1CCJEly/panic.csv
unit_tests/GFfQbPv6Lv3BvKTQsb6lvo1Yrs9rebvj3tVpWD8tJ0GtifAJLpAoX0l8x1CCJEly/edges_metadata.csv
unit_tests/37u9rWSWaXRP8OsN6qzlzuYTLJFxi8LIktAUoy12g2JHZzqbNSnueiURyJq0Fj4f/graph_metadata.csv
unit_tests/37u9rWSWaXRP8OsN6qzlzuYTLJFxi8LIktAUoy12g2JHZzqbNSnueiURyJq0Fj4f/panic.csv
unit_tests/37u9rWSWaXRP8OsN6qzlzuYTLJFxi8LIktAUoy12g2JHZzqbNSnueiURyJq0Fj4f/edges_metadata.csv
unit_tests/d26jJTNPorPvU5piiQlXhE93EJaTdoxVgLc6YtQyBWYlrAiluwwLhH8sTqg0aM8R/graph_metadata.csv
unit_tests/d26jJTNPorPvU5piiQlXhE93EJaTdoxVgLc6YtQyBWYlrAiluwwLhH8sTqg0aM8R/panic.csv
unit_tests/d26jJTNPorPvU5piiQlXhE93EJaTdoxVgLc6YtQyBWYlrAiluwwLhH8sTqg0aM8R/edges_metadata.csv
unit_tests/lhuavVb1BQnp8yKrxesRhWIETzw20CjwaF4c8U7v27wl6SAXLSu0WNxz3YC7Rlo1/graph_metadata.csv
unit_tests/lhuavVb1BQnp8yKrxesRhWIETzw20CjwaF4c8U7v27wl6SAXLSu0WNxz3YC7Rlo1/p

...

unit_tests/hVAJTAFKdrnpisl2tk6x7fGvuxelhyIRVyCIwTOk12JdDGZunEE14Xl2bxBmA1UH/edges_metadata.csv
unit_tests/KeCWp2GDmobFu90T4hisNo5kNK4F2xhaL60gJt7sqXzmqYBvwRyZnm7kHQUpBt6C/graph_metadata.csv
unit_tests/KeCWp2GDmobFu90T4hisNo5kNK4F2xhaL60gJt7sqXzmqYBvwRyZnm7kHQUpBt6C/panic.csv
unit_tests/KeCWp2GDmobFu90T4hisNo5kNK4F2xhaL60gJt7sqXzmqYBvwRyZnm7kHQUpBt6C/edges_metadata.csv
unit_tests/MwGzN8Gp1mDbtDJHbbMAEGd4DAdrq1gGg87cEOOIRjIlfFmdNWX6YoAPS9jEKGS4/graph_metadata.csv
unit_tests/MwGzN8Gp1mDbtDJHbbMAEGd4DAdrq1gGg87cEOOIRjIlfFmdNWX6YoAPS9jEKGS4/panic.csv
unit_tests/MwGzN8Gp1mDbtDJHbbMAEGd4DAdrq1gGg87cEOOIRjIlfFmdNWX6YoAPS9jEKGS4/edges_metadata.csv
unit_tests/045htmIIDVreNgWVJwXoMJ45hdn9ljm80gSv1h1jJ91Z3Y6PaNGhmPNKkkG9rZYN/graph_metadata.csv
unit_tests/045htmIIDVreNgWVJwXoMJ45hdn9ljm80gSv1h1jJ91Z3Y6PaNGhmPNKkkG9rZYN/panic.csv
unit_tests/045htmIIDVreNgWVJwXoMJ45hdn9ljm80gSv1h1jJ91Z3Y6PaNGhmPNKkkG9rZYN/edges_metadata.csv
unit_tests/9IKw2cVabrF9RPlxDOSYEwY2a3WHft84uEYNJws9XRPOhyXxvIQTRRC8pYEkxrxr/g

unit_tests/jZCWpjBmfhBO41LeozAdSZPSvma9vhnK2ilrz2pdd1O48gTEsMQwVcmFXLKNZmUx/graph_metadata.csv
unit_tests/jZCWpjBmfhBO41LeozAdSZPSvma9vhnK2ilrz2pdd1O48gTEsMQwVcmFXLKNZmUx/panic.csv
unit_tests/jZCWpjBmfhBO41LeozAdSZPSvma9vhnK2ilrz2pdd1O48gTEsMQwVcmFXLKNZmUx/edges_metadata.csv
unit_tests/zmZCmSG1SaD6gKX6tzmITg3G77w1tLlqSvber8aoh4Um0Iu7vh6wYjGnrh6bliqb/graph_metadata.csv
unit_tests/zmZCmSG1SaD6gKX6tzmITg3G77w1tLlqSvber8aoh4Um0Iu7vh6wYjGnrh6bliqb/panic.csv
unit_tests/zmZCmSG1SaD6gKX6tzmITg3G77w1tLlqSvber8aoh4Um0Iu7vh6wYjGnrh6bliqb/edges_metadata.csv
unit_tests/tnCetJK7aPM1nqZ2Ww6PVmuMmfYIFw4gqeGrXE2gw2mRrhLWqlr5mPaF4TMJYegA/graph_metadata.csv
unit_tests/tnCetJK7aPM1nqZ2Ww6PVmuMmfYIFw4gqeGrXE2gw2mRrhLWqlr5mPaF4TMJYegA/panic.csv
unit_tests/tnCetJK7aPM1nqZ2Ww6PVmuMmfYIFw4gqeGrXE2gw2mRrhLWqlr5mPaF4TMJYegA/edges_metadata.csv
unit_tests/RVopRuTJTa2lAdj51WQ0baH0t91R11XRoh1hS1kMFq1mfEKmJenUDWLgL6PgYqEu/graph_metadata.csv
unit_tests/RVopRuTJTa2lAdj51WQ0baH0t91R11XRoh1hS1kMFq1mfEKmJenUDWLgL6PgYqEu/p

unit_tests/qXh6PcjwAjfFcmxk30P6lFYD2QiBheRjlTMSOpmLyWv9qA2TNl8KBqX7qC9Q6TMO/graph_metadata.csv
unit_tests/qXh6PcjwAjfFcmxk30P6lFYD2QiBheRjlTMSOpmLyWv9qA2TNl8KBqX7qC9Q6TMO/panic.csv
unit_tests/qXh6PcjwAjfFcmxk30P6lFYD2QiBheRjlTMSOpmLyWv9qA2TNl8KBqX7qC9Q6TMO/edges_metadata.csv
unit_tests/4dD0XEICeZwPJEQBLwkPbnEFVUBTwDIhO1sKdyccAhm7iiilaHb4Kqz6MVJex0pP/graph_metadata.csv
unit_tests/4dD0XEICeZwPJEQBLwkPbnEFVUBTwDIhO1sKdyccAhm7iiilaHb4Kqz6MVJex0pP/panic.csv
unit_tests/4dD0XEICeZwPJEQBLwkPbnEFVUBTwDIhO1sKdyccAhm7iiilaHb4Kqz6MVJex0pP/edges_metadata.csv
unit_tests/53Rzm7T4V0IOJ5AQp0DP2uNj7uxUCK0tgiht6HRRBo6avoyY2Vt54p1GKF0x3aCZ/graph_metadata.csv
unit_tests/53Rzm7T4V0IOJ5AQp0DP2uNj7uxUCK0tgiht6HRRBo6avoyY2Vt54p1GKF0x3aCZ/panic.csv
unit_tests/53Rzm7T4V0IOJ5AQp0DP2uNj7uxUCK0tgiht6HRRBo6avoyY2Vt54p1GKF0x3aCZ/edges_metadata.csv
unit_tests/wdEGkYdzqsYzX0r9mQdYy6DM4NFxYQpVxaGaYtFfeFzzdXssoFKJmGHH95SzSgor/graph_metadata.csv
unit_tests/wdEGkYdzqsYzX0r9mQdYy6DM4NFxYQpVxaGaYtFfeFzzdXssoFKJmGHH95SzSgor/p

unit_tests/2G7WB1UclN5xrZdwH1ztMRhVxIf9IG7S7JSpeiyIC8QPoI9GVMYMjvSvQBI71yFD/edges_metadata.csv
unit_tests/hwtwcSZrg6Uokojdc0G3hd9GsphrkELPCQDusbDvxWbTV00bu5HuYZfe9u2Jb2BT/graph_metadata.csv
unit_tests/hwtwcSZrg6Uokojdc0G3hd9GsphrkELPCQDusbDvxWbTV00bu5HuYZfe9u2Jb2BT/panic.csv
unit_tests/hwtwcSZrg6Uokojdc0G3hd9GsphrkELPCQDusbDvxWbTV00bu5HuYZfe9u2Jb2BT/edges_metadata.csv
unit_tests/rCLQySkZbU1ADPRYP2L0BWj2SGWQBzdK98wJRWOBB2zilrCXf2wztg8ygbtrISHy/graph_metadata.csv
unit_tests/rCLQySkZbU1ADPRYP2L0BWj2SGWQBzdK98wJRWOBB2zilrCXf2wztg8ygbtrISHy/panic.csv
unit_tests/rCLQySkZbU1ADPRYP2L0BWj2SGWQBzdK98wJRWOBB2zilrCXf2wztg8ygbtrISHy/edges_metadata.csv
unit_tests/Lz63UkDnVt6XLzijIj8znOwCbCywtfYdJqKZaEWHVKPrhBk4XS5YjEEyPIHiGi70/graph_metadata.csv
unit_tests/Lz63UkDnVt6XLzijIj8znOwCbCywtfYdJqKZaEWHVKPrhBk4XS5YjEEyPIHiGi70/panic.csv
unit_tests/Lz63UkDnVt6XLzijIj8znOwCbCywtfYdJqKZaEWHVKPrhBk4XS5YjEEyPIHiGi70/edges_metadata.csv
unit_tests/Ptu1I8n6X4GpODG9pWiV2HIqPwAFjwZaJw56rLRIt3sAR5H3fvIxKH6oE8e3MZ9C/g

unit_tests/CsCm2t80nPhU415AmLVb6IMJFLUUcIvfmGVxIrd5fnvp1mlS2ZIfdz9gwp8pm2jR/edges_metadata.csv
unit_tests/k8pSVrNJHgyevcUBbajuZFEVh5T4RI3Q8EkMDZCd4q9AgTxIS4hJecc3SeiTZvBi/graph_metadata.csv
unit_tests/k8pSVrNJHgyevcUBbajuZFEVh5T4RI3Q8EkMDZCd4q9AgTxIS4hJecc3SeiTZvBi/panic.csv
unit_tests/k8pSVrNJHgyevcUBbajuZFEVh5T4RI3Q8EkMDZCd4q9AgTxIS4hJecc3SeiTZvBi/edges_metadata.csv
unit_tests/DS7yo2DtmSMQmBTIAFocqRSyYLZpLiqPdoJuAWczbZdRMqi3LmzshoirVStOk5J2/graph_metadata.csv
unit_tests/DS7yo2DtmSMQmBTIAFocqRSyYLZpLiqPdoJuAWczbZdRMqi3LmzshoirVStOk5J2/panic.csv
unit_tests/DS7yo2DtmSMQmBTIAFocqRSyYLZpLiqPdoJuAWczbZdRMqi3LmzshoirVStOk5J2/edges_metadata.csv
unit_tests/YNMXWFb9tT6x29x1ozOQVLgYVCDXMzKQsgqPKMyxYG2J854chSvvIiqmk7T2WMkv/graph_metadata.csv
unit_tests/YNMXWFb9tT6x29x1ozOQVLgYVCDXMzKQsgqPKMyxYG2J854chSvvIiqmk7T2WMkv/panic.csv
unit_tests/YNMXWFb9tT6x29x1ozOQVLgYVCDXMzKQsgqPKMyxYG2J854chSvvIiqmk7T2WMkv/edges_metadata.csv
unit_tests/1IavLJ7FvaMTdvsLQUgkxxfzPWhRc65CXox40TPdw4rEFUu2K8OPtRItqFwKFSrm/g

In [8]:
path = "../graph/tests/data/regression/100.edges"
values = pd.read_csv(path, sep=",", header=None)

FileNotFoundError: [Errno 2] No such file or directory: '../graph/tests/data/regression/100.edges'

In [4]:
remap = list(set(values[[0, 1, 2]].values.flatten()))
remapped_values = values.applymap(lambda x: remap.index(x) if x in remap else x)
remapped_values.to_csv(path, header=False, index=False)